<a href="https://colab.research.google.com/github/kandgesid/Data-Science-and-ML-projects/blob/master/cat_in_dat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df_train = pd.read_csv("/kaggle/input/cat-in-the-dat/train.csv")
df_test = pd.read_csv("/kaggle/input/cat-in-the-dat/test.csv")

In [ ]:
df_train.shape
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_target = df_train.pop('target')
train_id = df_train.pop("id")
test_id = df_test.pop("id")

In [ ]:
df_test.shape

Making copy of original data

In [ ]:
copy_df_train = df_train.copy()
copy_df_valid = df_test.copy()

collecting all categorical col

In [ ]:
s = (df_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

In [ ]:
object_nunique = list(map(lambda col: df_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))
#sorting in asceding order
sorted(d.items(), key=lambda x: x[1])

In [ ]:
low_cardinality_cols = [col for col in object_cols if df_train[col].nunique() < 15]

# Columns that will be dropped from the dataset
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)

In [ ]:
# we have apply label encoding on high caedinality
# Columns that can be safely label encoded
good_label_cols = [col for col in high_cardinality_cols if 
                   set(df_train[col]) == set(df_test[col])]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(high_cardinality_cols)-set(good_label_cols))
        
print('Categorical columns that will be label encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Drop categorical columns that will not be encoded
df_train = df_train.drop(bad_label_cols, axis=1)
df_test = df_test.drop(bad_label_cols, axis=1)

# Apply label encoder
label_encoder = LabelEncoder()
for col in set(good_label_cols):
    df_train[col] = label_encoder.fit_transform(df_train[col])
    df_test[col] = label_encoder.transform(df_test[col])

In [ ]:
df_test.shape

In [ ]:
df_train.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(df_train[low_cardinality_cols]))
OH_cols_test = pd.DataFrame(OH_encoder.transform(df_test[low_cardinality_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = df_train.index
OH_cols_test.index = df_test.index

# Remove low cardinality categorical columns (will replace with one-hot encoding)
num_df_train = df_train.drop(low_cardinality_cols, axis=1)
num_df_test = df_test.drop(low_cardinality_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_df_train = pd.concat([num_df_train, OH_cols_train], axis=1)
OH_df_test = pd.concat([num_df_test, OH_cols_test], axis=1)

In [ ]:
OH_df_train.shape

In [ ]:
OH_df_train.shape
#OH_df_test.head()
OH_df_train.drop(["bin_0"], axis=1, inplace=True)
OH_df_test.drop(["bin_0"], axis=1, inplace=True)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

"""def logistic(X,y):
    X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.2)
    lr=LogisticRegression()
    lr.fit(OH_df_train,df_target)
    y_pre=lr.predict(df_test)
    print('Accuracy : ',accuracy_score(y_test,y_pre))"""
lr = LogisticRegression(C=0.14,solver="lbfgs",max_iter=5000)

lr.fit(OH_df_train, df_target)

pred = lr.predict_proba(OH_df_train)[:, 1]
score = roc_auc_score(df_target,pred)

print(score)

In [ ]:
OH_df_test.head()

In [ ]:
submission = pd.DataFrame()
submission["id"] = test_id
submission["target"] = lr.predict_proba(OH_df_test)[:, 1]

In [ ]:
submission.head()